In [17]:
# =============================================================
# BLOCK A — ML DATA CLEANING PIPELINE (Pandas-Safe Version)
# Dataset: HealthCareAnalytics.csv
# =============================================================

import pandas as pd
import numpy as np
import re

# -------------------------------------------------------------
# 1. LOAD DATA
# -------------------------------------------------------------
df = pd.read_csv("HealthCareAnalytics.csv")
print("Initial shape:", df.shape)
df.head()



Initial shape: (318438, 18)


,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available_Extra_Rooms_in_Hospital,Department,Ward_Type,Ward_Facility_Code,Bed_Grade,patientid,City_Code_Patient,Type_of_Admission,Severity_of_Illness,Visitors_with_Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558,41-50


In [5]:
# -------------------------------------------------------------
# 2. REMOVE DUPLICATES
# -------------------------------------------------------------
duplicate_count = df.duplicated().sum()
df = df.drop_duplicates()
print("Removed duplicates:", duplicate_count)

Removed duplicates: 0


In [29]:
# -------------------------------------------------------------
# 3. HANDLE MISSING VALUES (with safe assignment)
# -------------------------------------------------------------

df = df.replace(["", " ", "  ", "NA", "nan", "None"], np.nan)

# Convert numeric-like columns to numeric
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df[col] = df[col].fillna(df[col].median())  # SAFE

# Categorical missing values → mode
cat_cols = df.select_dtypes(include=["object"]).columns
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])  # 

# Check missing values
df.isnull().sum()


case_id                              0
Hospital_code                        0
City_Code_Hospital                   0
Available_Extra_Rooms_in_Hospital    0
Bed_Grade                            0
patientid                            0
City_Code_Patient                    0
Severity_of_Illness                  0
Visitors_with_Patient                0
Age                                  0
Admission_Deposit                    0
Stay                                 0
dtype: int64

In [19]:
# -------------------------------------------------------------
# 4. CLEAN TEXT + FIX DATA TYPES
# -------------------------------------------------------------

def clean_text(x):
    return re.sub(r"[^A-Za-z0-9\- ]", "", str(x)).strip()

for col in cat_cols:
    df[col] = df[col].apply(clean_text)

# Convert range columns (e.g., "41-50") → numeric lower bound
def convert_range(x):
    try:
        return int(str(x).split("-")[0])
    except:
        return np.nan

if "Age" in df.columns:
    df["Age"] = df["Age"].apply(convert_range)
    df["Age"] = df["Age"].fillna(df["Age"].median())

if "Stay" in df.columns:
    df["Stay"] = df["Stay"].apply(convert_range)
    df["Stay"] = df["Stay"].fillna(df["Stay"].median())

# Ordinal encoding
if "Severity_of_Illness" in df.columns:
    sev_map = {"Minor": 1, "Moderate": 2, "Extreme": 3}
    df["Severity_of_Illness"] = df["Severity_of_Illness"].map(sev_map)
    df["Severity_of_Illness"] = df["Severity_of_Illness"].fillna(2) # assume Moderate as default
# Spot-check values
df.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available_Extra_Rooms_in_Hospital,Department,Ward_Type,Ward_Facility_Code,Bed_Grade,patientid,City_Code_Patient,Type_of_Admission,Severity_of_Illness,Visitors_with_Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,2.0,2,510,4911,0
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,2.0,2,510,5954,410
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,2.0,2,510,4745,310
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,2.0,2,510,7272,410
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,2.0,2,510,5558,410


In [20]:
# -------------------------------------------------------------
# 5. OUTLIER HANDLING (IQR capping)
# -------------------------------------------------------------

def cap_outliers(df_local, col):
    Q1 = df_local[col].quantile(0.25)
    Q3 = df_local[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df_local[col] = df_local[col].clip(lower, upper)

for col in ["Admission_Deposit", "Visitors_with_Patient"]:
    if col in df.columns:
        cap_outliers(df, col)
# Spot-check values
df.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available_Extra_Rooms_in_Hospital,Department,Ward_Type,Ward_Facility_Code,Bed_Grade,patientid,City_Code_Patient,Type_of_Admission,Severity_of_Illness,Visitors_with_Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,2.0,2,510,4911.0,0
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,2.0,2,510,5954.0,410
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,2.0,2,510,4745.0,310
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,2.0,2,510,7243.5,410
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,2.0,2,510,5558.0,410


### Outlier for Admission_Deposit and Visitors_with_Patient

In [22]:
# -------------------------------------------------------------
# 6. ONE-HOT ENCODING (SAFE)
# -------------------------------------------------------------
one_hot_cols = [
    "Type_of_Admission",
    "Department",
    "Ward_Type",
    "Ward_Facility_Code",
    "Hospital_type_code",
    "Hospital_region_code"
]

cols_to_encode = [c for c in one_hot_cols if c in df.columns]

df = pd.get_dummies(df, columns=cols_to_encode, drop_first=True)
# Spot-check values
df.head()

,case_id,Hospital_code,City_Code_Hospital,Available_Extra_Rooms_in_Hospital,Bed_Grade,patientid,City_Code_Patient,Severity_of_Illness,Visitors_with_Patient,Age,...,Ward_Facility_Code_E,Ward_Facility_Code_F,Hospital_type_code_b,Hospital_type_code_c,Hospital_type_code_d,Hospital_type_code_e,Hospital_type_code_f,Hospital_type_code_g,Hospital_region_code_Y,Hospital_region_code_Z
0,1,8,3,3,2.0,31397,7.0,2.0,2,510,...,False,True,False,True,False,False,False,False,False,True
1,2,2,5,2,2.0,31397,7.0,2.0,2,510,...,False,True,False,True,False,False,False,False,False,True
2,3,10,1,2,2.0,31397,7.0,2.0,2,510,...,True,False,False,False,False,True,False,False,False,False
3,4,26,2,2,2.0,31397,7.0,2.0,2,510,...,False,False,True,False,False,False,False,False,True,False
4,5,26,2,2,2.0,31397,7.0,2.0,2,510,...,False,False,True,False,False,False,False,False,True,False


### What One-Hot Encoding Does
One-hot encoding is a way to convert categorical variables (text labels) into numeric columns so that machine learning models can use them.

Categorical columns (like "Type_of_Admission" or "Department") contain string values such as "Emergency", "Trauma", "Radiotherapy".

ML algorithms don’t understand text directly — they need numbers.

One-hot encoding creates new binary columns (0/1) for each category.

In [25]:
# -------------------------------------------------------------
# REMOVE ANY TRUE/FALSE VALUES — STRICT MODE
# -------------------------------------------------------------

# Identify columns containing any True/False value
true_false_cols = []
rows_with_tf = pd.DataFrame()

for col in df.columns:
    col_lower = df[col].astype(str).str.lower()
    mask = col_lower.isin(["true", "false"])
    
    if mask.any():
        true_false_cols.append(col)
        rows_with_tf = pd.concat([rows_with_tf, df[mask]])

print("Columns containing True/False:", true_false_cols)

# 1. Drop columns that contain only True/False (pure booleans)
pure_bool_cols = [
    col for col in true_false_cols
    if set(df[col].astype(str).str.lower().unique()).issubset({"true", "false"})
]

df = df.drop(columns=pure_bool_cols, errors="ignore")
print("Dropped pure boolean columns:", pure_bool_cols)

# 2. Remove rows where TRUE/FALSE appears in any column
# Remove only on columns that still exist in the dataframe
for col in true_false_cols:
    if col in df.columns:
        df = df[~df[col].astype(str).str.lower().isin(["true", "false"])]


print("Shape after removing TRUE/FALSE contamination:", df.shape)

# Spot-check values
df.head()


Columns containing True/False: []
Dropped pure boolean columns: []
Shape after removing TRUE/FALSE contamination: (318438, 12)


,case_id,Hospital_code,City_Code_Hospital,Available_Extra_Rooms_in_Hospital,Bed_Grade,patientid,City_Code_Patient,Severity_of_Illness,Visitors_with_Patient,Age,Admission_Deposit,Stay
0,1,8,3,3,2.0,31397,7.0,2.0,2,510,4911.0,0
1,2,2,5,2,2.0,31397,7.0,2.0,2,510,5954.0,410
2,3,10,1,2,2.0,31397,7.0,2.0,2,510,4745.0,310
3,4,26,2,2,2.0,31397,7.0,2.0,2,510,7243.5,410
4,5,26,2,2,2.0,31397,7.0,2.0,2,510,5558.0,410


In [28]:
# -------------------------------------------------------------
# SAVE FINAL CLEANED DATASET
# -------------------------------------------------------------

output_path = "cleaned_hospital_stay.csv"
df.to_csv(output_path, index=False)

print(f"Cleaned dataset saved successfully as: {output_path}")
print("Location: same folder as this notebook")


Cleaned dataset saved successfully as: cleaned_hospital_stay.csv
Location: same folder as this notebook


# Summary of ML Data Cleaning (Block A)

### 1. Loaded Dataset
Loaded `HealthCareAnalytics.csv` into the notebook.

### 2. Removed Duplicates
Eliminated duplicate rows to avoid data leakage and bias.

### 3. Handled Missing Values
- Replaced invalid strings with NaN.
- Numeric columns → median imputation (robust to outliers).
- Categorical columns → mode imputation.

### 4. Fixed Data Types
- Cleaned categorical text by removing symbols.
- Converted range values (e.g., “41–50”) in Age and Stay into numeric lower bounds.
- Encoded Severity_of_Illness as an ordinal variable (Minor < Moderate < Extreme).

### 5. Outlier Handling
Applied IQR capping to:
- Admission_Deposit  
- Visitors_with_Patient  

### 6. Encoded Categorical Variables
Used one-hot encoding for:
Type_of_Admission, Department, Ward_Type, Ward_Facility_Code,  
Hospital_type_code, Hospital_region_code.

### 7. Saved Cleaned Dataset
Final ML-ready dataset exported as:
.......
cleaned_hospital_stay.csv
......

### Reasoning
All steps ensure the dataset contains:
- Only valid, numeric features suitable for ML  
- No missing values  
- No dangerous outliers  
- Clean categorical representations  
- Reliable target variable for model training  

Dataset is now ready for:
- Block B: EDA  
- Block C: Model Training


### Expected final ML modeling columns

After removing ID columns, this is the ML-ready feature set:

Hospital_code
city_code_hospital
available_extra_rooms_in_hospital
bed_grade
city_code_patient
severity_of_illness
visitors_with_patient
age
admission_deposit


And the Target for this project is : stay